In [1]:
import json
import os
import pandas as pd
from pycocotools.coco import COCO

In [5]:
# submission 양식에 맞게 output 후처리
def json_to_submission(json_file_path, submission_path, output_dir, output_file_name, score_thr):

    with open(json_file_path, 'r') as f:
        data = json.load(f)

    coco = COCO(submission_path)
    img_ids = coco.getImgIds()

    prediction_strings = ['' for _ in range(len(img_ids))]
    file_names = [f'test/{str(img_id).zfill(4)}.jpg' for img_id in range(len(img_ids))]

    for info in data:

        img_id = info['image_id']
        xmin, ymin, w, h = info['bbox']
        xmax, ymax = xmin+w, ymin+h
        score = info['score']
        cate_id = info['category_id']

        if score < score_thr:
            continue

        prediction_list = map(str, [cate_id, score, xmin, ymin, xmax, ymax])
        prediction_string = ' '.join(prediction_list) + ' '

        prediction_strings[img_id] += prediction_string


    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names

    os.makedirs(output_dir, exist_ok=True)
    submission.to_csv(os.path.join(output_dir, output_file_name), index=None)
    print(submission.head())
    print(submission.tail())

In [6]:
json_file_path = './mmdetection/work_dirs/ddq_recycle/result/ddq_w_tta.bbox.json'
submission_path = './dataset/test.json'
output_path = './submission'
output_file_name = 'ddq_result_tta_0.1.csv'
score_thr = 0.1

json_to_submission(json_file_path, submission_path, output_path, output_file_name, score_thr)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.8206961154937744 602.621826171875 518.2069...  test/0000.jpg
1  4 0.7349591255187988 342.415283203125 251.6196...  test/0001.jpg
2  1 0.5537089109420776 299.5743103027344 321.195...  test/0002.jpg
3  9 0.7441387176513672 148.0640869140625 263.252...  test/0003.jpg
4  1 0.7243448495864868 198.70361328125 256.79827...  test/0004.jpg
                                       PredictionString       image_id
4866  1 0.7725503444671631 296.81365966796875 348.37...  test/4866.jpg
4867  3 0.714236855506897 415.2959289550781 319.4771...  test/4867.jpg
4868  7 0.762602686882019 120.55071258544922 438.857...  test/4868.jpg
4869  0 0.6993628740310669 3.29095458984375 235.4173...  test/4869.jpg
4870  4 0.7853445410728455 372.771484375 199.7835388...  test/4870.jpg


In [132]:
submission

,PredictionString,image_id
0,7 0.8840062022209167 603.3726196289062 519.477...,test/0000.jpg
1,5 0.6590912938117981 344.5780029296875 249.323...,test/0001.jpg
2,1 0.608161449432373 666.5484008789062 225.8659...,test/0002.jpg
3,9 0.8630878329277039 142.63284301757812 262.64...,test/0003.jpg
4,1 0.6815583109855652 188.62750244140625 252.41...,test/0004.jpg
...,...,...
4866,0 0.6311280727386475 292.06866455078125 345.97...,test/4866.jpg
4867,3 0.7723613977432251 414.0743408203125 321.060...,test/4867.jpg
4868,7 0.8247560858726501 117.16061401367188 451.65...,test/4868.jpg
4869,0 0.7465888857841492 7.88055419921875 241.0235...,test/4869.jpg
